In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
# replace this with your picturedRocks that was included in this repo
import sys
sys.path.append('/home/ahsvargo/xvalid')

In [4]:
from picturedrocks import Rocks
from picturedrocks.performance import FoldTester, PerformanceReport, NearestCentroidClassifier

In [5]:
import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode

In [6]:
import plotly.io as pio

In [7]:
init_notebook_mode(connected=True)

In [8]:
import scanpy.api as sc
import anndata

/home/ahsvargo/miniconda3/envs/r35py37/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning:


examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.



In [9]:
import loadData as ld

scanpy==1.3.7+56.gde16b79.dirty anndata==0.6.11 numpy==1.14.6 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.0 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


## Random Forest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
class RandomForest:
    def __init__(self):
        self.traindata = None
        self.RFC = RandomForestClassifier(n_estimators=100, n_jobs=-1)
        
    def train(self, data):
        self.traindata = data
        self.RFC.fit(data.X, data.y[:,0])
        
    def test(self, Xtest, sparse):
        return self.RFC.predict(Xtest)
    

## Load data

Also need to change these paths in order to load the data

In [12]:
path= "/home/ahsvargo/turbo/scData/zheng17/filtered_matrices_mex/hg19/"

In [13]:
allGenes=True
labels='bulk'
adata, folds = ld.load_zheng('adata', labels=labels, allGenes=allGenes)

Return object contains bulk labels under obs.('bulk_labels')
and louvain labels under obs.('louv_labels')


In [14]:
bulk_lookup = list(np.unique(adata.obs['bulk_labels'].values))
louv_lookup = [str(a) for a in np.unique(adata.obs['louv_labels'].values)]

In [15]:
bulky = np.array([bulk_lookup.index( adata.obs['bulk_labels'][i] ) for i in range(adata.obs['bulk_labels'].shape[0]) ]) 
louvy = np.load(path + 'zheng17_yVec_lvals.npz')['y']

In [16]:
labels = "bulk"
sc.pp.normalize_per_cell(adata, counts_per_cell_after=10000)
sc.pp.log1p( adata )

data = Rocks(adata.X, bulky)
ft = FoldTester(data)

ft.folds = folds
ft.validatefolds()

ft.makerocks(1)

0

In [14]:
dataset = "zheng"
if not allGenes: dataset += "Filt"
path = "../zheng/plot"
patht = "/home/ahsvargo/turbo/scData/zheng17/filtered_matrices_mex/hg19/"

In [20]:
adata.var.index

Index(['AL627309.1', 'AP006222.2', 'RP11-206L10.3', 'RP11-206L10.2',
       'RP11-206L10.9', 'FAM87B', 'LINC00115', 'FAM41C', 'RP11-54O7.1',
       'SAMD11',
       ...
       'AC145212.1', 'MGC39584', 'AC011043.1', 'ZNF84-1', 'AL592183.1',
       'AC011841.1', 'AL354822.1', 'KIR2DL2', 'PNRC2-1', 'SRSF10-1'],
      dtype='object', name='index', length=20387)

## Load markers

Only scanpy methods right now (April 30 2019)

May 09 2019 added edgeR and MAST (without the det versions)

June 09 2019 These paths won't work in relation to the repo.  You can't really load the markers as written - see further below in the notebook for how to process the classification results (the yhats) or copy this information with the correct paths to play with the markers.

In [15]:
methods = ['wilcoxon', 't-test_overestim_var', 'logreg', 'edgeR', 'MAST']
method = methods[3]

In [30]:
def load_markers(method, dataset, path, labels=""):
    
    # scanpy methods
    if method in ['wilcoxon', 't-test_overestim_var', 'logreg']:
        fname = path + dataset + "-" + method + "-markerList"
        if labels is "":
            fname += ".npz"
        else:
            fname += "-" + labels + ".npz"
        
        print("Loading markers from " + fname)
        
        pvalMarkerList = np.load(fname)['marks']
        pvalScoreList = np.load(fname)['pvals']
       
    # R based methods
    elif method in ['edgeR', 'edgeRdet', 'MAST', 'MASTdet']:

        if 'zheng' in dataset and method in ['edgeRdet', 'MASTdet']:
            print("Error: R det methods only run on datasets 'paul15' and 'zeisel'")
            return
        
        pmethods = ["edgeR/", 'edgeRdet/', 'MASTtpm/', 'MASTtpmDet/']
        zmethods = ["edgeR/", 'edgeRdet/', 'MAST/', 'MASTdet/']
        zhmethods = ['edgeR/', 'q', 'mast/', 'q']
        ind = ["edgeR", 'edgeRdet', 'MAST', 'MASTdet'].index(method)
        
        
        
        if dataset is "paul15":
            method  = pmethods[ind]
        elif dataset is "zeisel":
            method = zmethods[ind]
        elif "zheng" in dataset:
            method = zhmethods[ind]
            
        pvalMarkerList = []
        for fold in range(5):
            dayta = np.load(path + method + "markerList-fold" + str(fold) + ".npz")
            pvalMarkerList.append(dayta['arr_0'])
            
    # random markers
    elif method is "random":
        pvalMarkerList = np.load(dataset + "-randomMarkers.npz")['arr_0']
        
    # scvi    
    elif method is "scvi":
        path="./scvi/"
        pvalMarkerList = []
        for fold in range(5):
            dayta = np.load(path + dataset + "-scvi-marks-fold" + str(fold) + ".npz")
            pvalMarkerList.append(dayta['arr_0'])
            
    # elastic nets
    elif method is "enets":
        path = "./enets/"
        pvalMarkerList = []
        for fold in range(5):
            dayta = np.load(path + dataset + "-nets-fold" + str(fold) + "-marks.npz")
            pvalMarkerList.append(dayta['arr_0'])
            
    else: 
        print("Error: Method must be one of the following:")
        print("'wilcoxon', 't-test_overestim_var', 'logreg', 'edgeR', 'edgeRdet', 'MAST', 'MASTdet', 'random', 'scvi', 'enets'")
        print("Return None")
        return None
    
    return pvalMarkerList

In [20]:
methods = ['wilcoxon', 't-test_overestim_var', 'logreg']
method = methods[2]

In [97]:
labels = 'bulk'
pvalMarkerList = load_markers(method, dataset=dataset, path=path, labels=labels)

Loading markers from zheng/zhengFilt-logreg-markerList-bulk.npz


In [76]:
np.array(pvalMarkerList).shape

(5, 11, 5000)

In [88]:
marks_per_clust=15
[len(a) for a in [list(set().union(*[marks[:marks_per_clust] for marks in table])) for table in pvalMarkerList]]

[606, 596, 593, 592, 588]

In [99]:
marks_list_test = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,35,40,45,50,55,60,65,70,75]

testXvals = [ np.array(
        [ len(a) for a in 
            [list(set().union(*[marks[:marks_per_clust] for marks in table])) 
            for table in pvalMarkerList]
        ]
        ).mean() for marks_per_clust in marks_list_test]

np.array(testXvals)

array([ 18.8,  27.2,  36.2,  45.2,  53.8,  60.4,  69.6,  78.4,  87.2,
       104.2, 121.4, 137.8, 154.8, 173. , 214.6, 255.4, 295.2, 332.4,
       368.6, 406.4, 442. , 477. , 513.6, 548.2, 581.8])

## Classify using the markers

In [ ]:
#ylist.append(perf.wrong())
#ylist.append(skm.recall_score(data.y.astype(int), ft.yhat, average='weighted'))

In [100]:
method

'logreg'

In [101]:
%%time
#zheng bulk labels
marks_list_wc = [2,4,6,10,14,30,48,51,54,57,60,66,72,78,84,100,115,130,145,160,175,190]
marks_list_wc5 = [2,4,6,8,10,12,16,20,24,30,36,44,52,60,68,90,112,130,144,166,188,200]
marks_list_wc5rf = [2,4,6,8,10,12,16,20,24,30,36,44,52,60,68,80,92,104,128,152,176,200]
marks_list_tt = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,58,70,82,94,106,118,130]
marks_list_tt5 = [2,4,6,8,10,12,14,16,20,25,30,35,40,48,56,68,80,94,110,126,142,156]
marks_list_lr = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,35,40,45,50,55,60]
marks_list_lr5 = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,35,40,45,50,55,60,65,70,75]
marks_list_mast5 = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,60,70,80,90,100,110,120,130]
marks_list_edge5 = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,60,70,80,90,100,110,120,130]
#zheng louv labels
#marks_list_wc = [2,4,6,10,14,18,22,26,30,39,48,51,54,57,60,66,72,78,84,100,115,130,145,160,175]
#marks_list_wc5 = [2,4,6,8,10,12,16,20,24,30,36,44,52,60,68,90,112,130,144,166,188,210]
#marks_list_tt = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,58,70,82,94,106,118,130,142,150]
#marks_list_tt5 = [2,4,6,8,10,12,14,16,20,25,30,35,40,48,56,68,80,94,110,126,142,156]
#marks_list_lr = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,35,40,45,50,55,60,65]

marks_list = marks_list_lr5

xvals = []
yhats = []

for marks_per_clust in marks_list:
    
    print("markers per cluster: {}".format(marks_per_clust), flush=True)
    # every method other than elastic nets:
    #myMarks = [list(set().union(*table[:,:marks_per_clust])) for table in pvalMarkerList]
    # elastic nets:
    myMarks = [list(set().union(*[marks[:marks_per_clust] for marks in table])) for table in pvalMarkerList]
    ft.markers = myMarks
    
    print("Number of markers per fold: {}".format([len(a) for a in ft.markers]))
    xvals.append([len(a) for a in ft.markers])
    
    ft.classify(RandomForest)
    #perf = PerformanceReport(data.y.astype(int), ft.yhat)
    yhats.append(ft.yhat)
    

markers per cluster: 2
Number of markers per fold: [18, 18, 19, 20, 19]
markers per cluster: 3
Number of markers per fold: [26, 29, 26, 27, 28]
markers per cluster: 4
Number of markers per fold: [35, 38, 36, 35, 37]
markers per cluster: 5
Number of markers per fold: [43, 47, 45, 44, 47]
markers per cluster: 6
Number of markers per fold: [52, 56, 55, 52, 54]
markers per cluster: 7
Number of markers per fold: [59, 61, 62, 59, 61]
markers per cluster: 8
Number of markers per fold: [69, 70, 71, 68, 70]
markers per cluster: 9
Number of markers per fold: [79, 80, 80, 76, 77]
markers per cluster: 10
Number of markers per fold: [87, 90, 88, 84, 87]
markers per cluster: 12
Number of markers per fold: [107, 106, 106, 101, 101]
markers per cluster: 14
Number of markers per fold: [125, 122, 121, 121, 118]
markers per cluster: 16
Number of markers per fold: [138, 141, 137, 138, 135]
markers per cluster: 18
Number of markers per fold: [155, 155, 154, 157, 153]
markers per cluster: 20
Number of marke

In [102]:
np.array([np.array(a).mean() for a in xvals])

array([ 18.8,  27.2,  36.2,  45.2,  53.8,  60.4,  69.6,  78.4,  87.2,
       104.2, 121.4, 137.8, 154.8, 173. , 214.6, 255.4, 295.2, 332.4,
       368.6, 406.4, 442. , 477. , 513.6, 548.2, 581.8])

In [103]:
errs = np.array(list( map(lambda yhat: adata.X.shape[0] - np.where(yhat == bulky)[0].shape[0], yhats) ))
errs

array([31403, 28040, 26309, 25254, 23612, 22843, 21804, 20712, 19749,
       18212, 17511, 17150, 16321, 16087, 15594, 15113, 14781, 14846,
       14740, 14579, 14634, 14544, 14646, 14444, 14586])

In [104]:
fname = "zhengFilt-" + method + "-" + labels + "-plotInfo-RF.npz"
print("Save data to " + fname + "?")

Save data to zhengFilt-logreg-bulk-plotInfo-RF.npz?


In [105]:
np.savez(fname, xvals=xvals, yhats=yhats, ytrue=bulky, marks=marks_list, errs=errs)

Below: They are the same file?

In [45]:
stuff = np.load("zhengFilt-rankCorr-bulk-plotInfo-RF.npz")
stuff['errs']

array([25805, 20776, 18403, 17367, 16879, 16550, 16293, 16086, 15429,
       15493, 15198, 15096, 15102, 15137, 15208, 15345, 15322, 15323,
       15340, 15242, 15421])

In [48]:
stuff = np.load("/home/ahsvargo/turbo/scData/zheng17/filtered_matrices_mex/hg19/zhengFilt-rankCorr-bulk-plotInfo-RF.npz")
stuff['errs']

array([25805, 20776, 18403, 17367, 16879, 16550, 16293, 16086, 15429,
       15493, 15198, 15096, 15102, 15137, 15208, 15345, 15322, 15323,
       15340, 15242, 15421])

In [50]:
np.array([np.array(a).mean() for a in stuff['xvals']])

array([ 11.6,  23.2,  33.4,  45.2,  59.8,  78. ,  95.8, 113.8, 140.8,
       168.4, 191.8, 222.4, 251.2, 282. , 316.4, 353.2, 388.4, 430.8,
       478. , 524.6, 568.2])

## Aggregate the data and compute the values mentioned in the paper.

In [21]:
import sklearn.metrics as skm

In [20]:
methods = ['wilcoxon', 't-test_overestim_var', 'logreg']

In [22]:
meths = ['rankCorr'] + methods

In [23]:
meths

['rankCorr', 'wilcoxon', 't-test_overestim_var', 'logreg']

In [24]:
filtMeths = ['rankCorr'] + methods + ['edgeR', 'MAST']

In [25]:
allGenes = False
labels = 'bulk'

if labels is 'bulk':
    yTrue = bulky
elif labels is 'louv':
    yTrue = louvy
    
if not allGenes and labels is 'bulk':
    currMeths = filtMeths
else:
    currMeths = meths

NameError: name 'bulky' is not defined

In [169]:
currMeths

['rankCorr', 'wilcoxon', 't-test_overestim_var', 'logreg', 'edgeR', 'MAST']

In [119]:
np.unique(bulky, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 3306,  3817,   262,    19, 14112,  2793,  3126,  5859, 11445,
        21975,  1865]))

In [170]:
dayta = []
accs = []
prec = []
recs = []
mccs = []
xvals = []

for method in currMeths:
    
    fname = "zheng"
    if not allGenes: fname += "Filt-"
    else: fname += "-"
    
    fname += method + "-" + labels + "-plotInfo.npz"
    print("PROCESSING DATA FROM {}".format(fname))
    
    dayta = np.load(fname) 
    
    accs.append( list(map(lambda yhat: 1-skm.accuracy_score(yTrue, yhat), dayta['yhats'])) )
    prec.append( list(map(lambda yhat: skm.precision_score(yTrue, yhat, average='weighted'), dayta['yhats'])) )
    recs.append( list(map(lambda yhat: skm.recall_score(yTrue, yhat, average='weighted'), dayta['yhats'])) )
    mccs.append( list(map(lambda yhat: skm.matthews_corrcoef(yTrue, yhat), dayta['yhats'])) )
    
    xvals.append( np.array([np.array(a).mean() for a in dayta['xvals']]) )
    
    dayta.close()
    
outname = "zheng"
if not allGenes: outname += "Filt-"
else: outname += "-"

outname += labels + "scores-all.npz"
print("Saving all processed data to {}".format(outname))

np.savez(outname, accs=accs, prec=prec, recs=recs, mccs=mccs, xvals=xvals)

PROCESSING DATA FROM zhengFilt-rankCorr-bulk-plotInfo.npz
PROCESSING DATA FROM zhengFilt-wilcoxon-bulk-plotInfo.npz
PROCESSING DATA FROM zhengFilt-t-test_overestim_var-bulk-plotInfo.npz
PROCESSING DATA FROM zhengFilt-logreg-bulk-plotInfo.npz
PROCESSING DATA FROM zhengFilt-edgeR-bulk-plotInfo.npz
PROCESSING DATA FROM zhengFilt-MAST-bulk-plotInfo.npz
Saving all processed data to zhengFilt-bulkscores-all.npz


## Plot the data

These plots should all run without needing to change any paths.  To make things work easily, head down to the end of the file and define `layout` before continuing here.

In [20]:
methods = ['wilcoxon', 't-test_overestim_var', 'logreg']

In [22]:
meths = ['rankCorr'] + methods

In [23]:
meths

['rankCorr', 'wilcoxon', 't-test_overestim_var', 'logreg']

In [24]:
filtMeths = ['rankCorr'] + methods + ['edgeR', 'MAST']

In [27]:
allGenes = False
labels = 'bulk'

if not allGenes and labels is 'bulk':
    currMeths = filtMeths
else:
    currMeths = meths

In [28]:
colorMap = {
    
    'rankCorr' : "#3366cc",
#    methods[0] : "#ef3774", # nice pink
    methods[0] : "#d62728", # red
    methods[1] : "#ff9900",
#    methods[2] : "#22aa99",  # green-blue (close to green)
    methods[2] : "#2ca02c",  #chalkboard
    'edgeR' :  "#9467bd",
    'edgeRdet' : "#c5b0d5",
    'MAST' : '#17becf',
    'MASTdet': '#9edae5',
    'enets': "#8c564b",
#    'genzel' : "#0099c6", # blueish
    'genzel' : "#bcbd22", #vom
    'scvi' : "#f032e6",
    'random' : '#808080'
    
}

In [29]:
path = '../zheng17/plot/'

In [30]:
score = 'accs'

daytaname = "zheng"
if not allGenes: daytaname += "Filt-"
else: daytaname += "-"
    
daytaname += labels + "scores-all.npz"
dayta = np.load(path + daytaname)

methNames = {
    'rankCorr': "RankCorr",
    methods[0]: "Wilcoxon",
    methods[1]: "t-test",
    methods[2]: "Log. Reg.",
    'edgeR' : 'edgeR',
    'MAST' : 'MAST'
}

traces = []
for i,method in enumerate(currMeths):
    traces.append(
        go.Scatter(x=dayta['xvals'][i], y=dayta[score][i], name=methNames[method], mode="lines+markers")
    )
    
    

In [34]:
iplot({'data':traces, 'layout': layout})

In [174]:
iplot({'data':traces, 'layout': layout})

In [176]:
iplot({'data':traces, 'layout': layout})

In [186]:
iplot({'data':traces, 'layout': layout1})

In [187]:
pio.write_json({'data': traces, 'layout': layout1}, "pics2/zhengfilt-nc-acc-bulk-all.json")

In [32]:
layout1 = go.Layout(
    colorway = [colorMap[meth] for meth in currMeths],
    font = dict(
            family='CMU Serif'
    ),
    showlegend=False,
    xaxis=dict(
        title='Number of markers',
        showgrid=False,
        ticks='inside',
        showline=True,
        mirror='ticks',
        #range=[10,170],
        range=[0,420],
        titlefont=dict(
            #family='Computer Modern',
            size=30,
            color='#000'
        ),
        tickfont=dict(
            size=24,
            color='#000'
        )
    ),
    yaxis=dict(
        title='Classification error rate',
        #title='Average precision',
        #title='Matthews correlation coefficient',
        showgrid=False,
        ticks='inside',
        showline=True,
        mirror='ticks',
        #range=[0.03,0.16],
        titlefont=dict(
            #family='Computer Modern',
            size=30,
            color='#000'
        ),
        tickfont=dict(
            size=24,
            color='#000'
        )
    ),
    legend=dict(
        #x=0,
        #y=1,
        #traceorder='normal',
        font=dict(
            #family='Computer Modern',
            size=20,
            #color='#000'
        ),
        #bgcolor='#E2E2E2',
        #bordercolor='#FFFFFF',
        #borderwidth=2
    ),
    margin=go.layout.Margin(
        l=100,
        r=10,
        b=90,
        t=10,
        #pad=4
    ),
)

In [33]:
layout = go.Layout(
    
    font = dict(
            family='CMU Serif'
    ),
    #showlegend=False,
    xaxis=dict(
        title='Number of markers',
        range=[0,550],
        titlefont=dict(
            #family='Computer Modern',
            size=20,
        ),
        tickfont=dict(
            size=16,
        )
    ),
    yaxis=dict(
        title='Matthews correlation coefficient',
        #title='Average precision',
        titlefont=dict(
            #family='Computer Modern',
            size=20,
        ),
        tickfont=dict(
            size=16,
        )
    ),
    legend=dict(
        #x=0,
        #y=1,
        #traceorder='normal',
        font=dict(
            #family='Computer Modern',
            size=16,
            #color='#000'
        ),
        #bgcolor='#E2E2E2',
        #bordercolor='#FFFFFF',
        #borderwidth=2
    ),
    margin=go.layout.Margin(
        #l=50,
        #r=50,
        #b=100,
        t=20,
        #pad=4
    ),
)